PIPES

In [ ]:
// pipes.c
#include <stdio.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <unistd.h>
int main() {
    int fd[2], n;
    char buffer[100];
    pid_t p;
    pipe(fd);
    p = fork();
    if (p > 0) {
        printf("Parent Passing value to child\n");
        write(fd[1], "this is the data\n", 6);
    } else {
        printf("Child printing received value\n");
        n = read(fd[0], buffer, 100);
        write(1, buffer, n);
    }
}
// Parent Passing value to child
// Child printing received value
// this i

FIFO

In [ ]:
// fifo.c
#include <stdio.h>
#include <sys/stat.h>
#include <sys/types.h>
int main() {
    int res = mkfifo("fifo1", 0777);
    printf("named pipe created\n");
}
// named pipe created

In [ ]:
// fifosend.c
#include <fcntl.h>
#include <stdio.h>
#include <unistd.h>
int main() {
    int res = open("fifo1", O_WRONLY);
    write(res, "message", 7);
    printf("sender process having pid %d sent the data\n", getpid());
}

In [ ]:
// fiforecv.c
#include <fcntl.h>
#include <stdio.h>
#include <unistd.h>
int main() {
    char buffer[100];
    int res = open("fifo1", O_RDONLY);
    int n = read(res, buffer, 100);
    printf("Reader process %d started\n", getpid());
    printf("Data received by receiver %d is: %s\n", getpid(), buffer);
}

// $ ./fsend && ./frecv
// [1] 5385
// Sender process having pid 5385 sent the data
// Reader process 5386 started
// Data received by receiver 5386 is: message
// [1]+  Done              ./fsend

MESSAGE QUEUE

In [ ]:
// send.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <sys/ipc.h>
#include <sys/msg.h>
#include <sys/types.h>
#include <unistd.h>
#define MAX_TEXT 512
struct my_msg {
    long int msg_type;
    char some_text[MAX_TEXT];
};
int main() {
    int running = 1;
    struct my_msg some_data;
    char buffer[50];
    int msgid = msgget((key_t)14534, 0666 | IPC_CREAT);
    if (msgid == -1) {
        printf("Error in creating queue\n");
        exit(0);
    }
    while (running) {
        printf("Enter some text:\n");
        fgets(buffer, 50, stdin);
        some_data.msg_type = 1;
        strcpy(some_data.some_text, buffer);
        if (msgsnd(msgid, (void *)&some_data, MAX_TEXT, 0) == -1) printf("Msg not sent\n");
        if (strncmp(buffer, "end", 3) == 0) running = 0;
    }
}
// $ ./send
// Enter some text:
// hello
// Enter some text:
// world
// Enter some text:
// end

In [ ]:
// recv.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <sys/ipc.h>
#include <sys/msg.h>
#include <sys/types.h>
#include <unistd.h>
struct my_msg {
    long int msg_type;
    char some_text[BUFSIZ];
};
int main() {
    int running = 1;
    struct my_msg some_data;
    long int msg_to_rec = 0;
    int msgid = msgget((key_t)14534, 0666 | IPC_CREAT);
    while (running) {
        msgrcv(msgid, (void *)&some_data, BUFSIZ, msg_to_rec, 0);
        printf("Data received: %s\n", some_data.some_text);
        if (strncmp(some_data.some_text, "end", 3) == 0) running = 0;
    }
    msgctl(msgid, IPC_RMID, 0);
}
// $ ./recv
// Data received: hello
// Data received: world
// Data received: end

SHARED MEMORY

In [ ]:
// send.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <sys/shm.h>
#include <unistd.h>
int main() {
    char buffer[100];
    int shmid = shmget((key_t)1985, 1024, 0666 | IPC_CREAT);
    printf("key of shared memory is : %d\n", shmid);
    void *shared_memory = shmat(shmid, NULL, 0);
    printf("process attached at %p\n", shared_memory);
    printf("enter some data to write to a shared memory \n");
    read(0, buffer, 100);
    strcpy(shared_memory, buffer);
    printf("your data is %s\n", (char *)shared_memory);
}
// $ ./send
// key of shared memory is : 32810
// process attached at 0x7f8b8f8b8f00
// enter some data to write to a shared memory
// hello world
// your data is hello world

In [ ]:
// recv.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <sys/shm.h>
#include <unistd.h>
int main() {
    char buffer[100];
    int shmid = shmget((key_t)1985, 1024, 0666);
    printf("key of shared memory is : %d\n", shmid);
    void *shared_memory = shmat(shmid, NULL, 0);
    printf("process attached at %p\n", shared_memory);
    printf("data read from shared memory is : %s\n", (char *)shared_memory);
}
// $ ./recv
// key of shared memory is : 32810
// process attached at 0x7f8b8f8b8f00
// data read from shared memory is : hello world

FCFS

In [ ]:
// fcfs.c
#include <stdio.h>
void swap(int arr[], int i, int j) {
    int temp = arr[i]; 
    arr[i] = arr[j];
    arr[j] = temp;
}
int main() {
    int n;
    printf("Enter number of processes : "); scanf("%d", &n);
    int arrid[n], arrat[n], arrb[n], ft[n], tat[n], wt[n];
    printf("Enter ids, arrival times and burst times : \n");
    for (int i = 0; i < n; i++) scanf("%d %d %d", &arrid[i], &arrat[i], &arrb[i]);
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            if (arrat[i] < arrat[j]) {
                swap(arrat, i, j);
                swap(arrid, i, j);
                swap(arrb, i, j);
            }
        }
    }
    ft[0] = arrat[0] + arrb[0];
    for (int i = 1; i < n; i++) ft[i] = ft[i - 1] + arrb[i];
    for (int i = 0; i < n; i++) {
        tat[i] = ft[i] - arrat[i];
        wt[i] = tat[i] - arrb[i];
    }
    int avgwt = 0, avgtat = 0;
    printf("id\t AT\t BT\t FT\t TAT\t WT\n");
    for (int i = 0; i < n; i++) {
        printf("%d\t %d\t %d\t %d\t %d\t %d\n", arrid[i], arrat[i], arrb[i], ft[i], tat[i], wt[i]);
        avgtat += tat[i];
        avgwt += wt[i];
    }
    printf("avg TAT : %d\n", avgtat / n);
    printf("avg WT : %d\n", avgwt / n);
}
// Enter number of processes : 5
// Enter ids, arrival times and burst times : 
// 0 0 80
// 1 0 20 
// 2 0 10
// 3 0 20
// 4 0 50
// id       AT      BT      FT      TAT     WT   
// 0        0       80      80      80      0    
// 1        0       20      100     100     80   
// 2        0       10      110     110     100  
// 3        0       20      130     130     110  
// 4        0       50      180     180     130  
// avg TAT : 120
// avg WT : 84

FITS

In [ ]:
// first.c
#include <stdio.h>
void main() {
    int bs, ps, bsize[10], psize[10], all[10], flag[10];
    printf("Enter the number of blocks: ");
    scanf("%d", &bs);
    printf("Enter the size of each block: ");
    for (int i = 0; i < bs; i++) scanf("%d", &bsize[i]);
    printf("Enter the number of processes: ");
    scanf("%d", &ps);
    printf("Enter the size of each process: ");
    for (int i = 0; i < ps; i++) scanf("%d", &psize[i]);
    for (int i = 0; i < ps; i++) {
        all[i] = -1;
        flag[i] = 0;
    }
    for (int i = 0; i < ps; i++) {
        for (int j = 0; j < bs; j++) {
            if (!flag[j] && bsize[j] >= psize[i]) {
                all[i] = j;
                flag[j] = 1;
                break;
            }
        }
    }
    printf("\nProcess No.\tProcess Size\tBlock no.\n");
    for (int i = 0; i < ps; i++) {
        printf("%d \t\t\t %d \t\t\t", i + 1, psize[i]);
        if (all[i] != -1) printf("%d\n", all[i] + 1);
        else printf("Not allpcated\n");
    }
}
// $ ./first
// Enter no of blocks : 5
// Enter the size of each block : 100 500 200 300 600
// Enter the no of processes : 4
// Enter the size of each process : 212 417 112 426
// 
// processno       process size    Block no
// 1               212             2
// 2               417             5
// 3               112             3
// 4               426             Not allocated

In [ ]:
// best.c
#include <stdio.h>
void main() {
    int fragment[20], b[20], p[20], i, j, nb, np, temp, lowest = 9999;
    static int barray[20], parray[20];
    printf("\n\t\t\tMemory Management Scheme - Best Fit");
    printf("\nEnter the number of blocks : ");
    scanf("%d", &nb);
    printf("Enter the number of processes : ");
    scanf("%d", &np);
    printf("\nEnter the size of the blocks : \n");
    for (i = 1; i <= nb; i++) {
        printf("Block no.%d : ", i);
        scanf("%d", &b[i]);
    }
    printf("\nEnter the size of the processes : \n");
    for (i = 1; i <= np; i++) {
        printf("Process no.%d : ", i);
        scanf("%d", &p[i]);
    }
    for (i = 1; i <= np; i++) {
        for (j = 1; j <= nb; j++) {
            if (barray[j] != 1) {
                temp = b[j] - p[i];
                if (temp >= 0) {
                    if (lowest > temp) {
                        parray[i] = j;
                        lowest = temp;
                    }
                }
            }
        }
        fragment[i] = lowest;
        barray[parray[i]] = 1;
        lowest = 10000;
    }
    printf("\nProcess_no\tProcess_size\tBlock_no\tBlock_size\tFragment");
    for (i = 1; i <= np && parray[i] != 0; i++)
        printf("\n%d\t\t%d\t\t%d\t\t%d\t\t%d", i, p[i], parray[i], b[parray[i]], fragment[i]);
}
// $ ./best
//                         Memory Management Scheme - Best Fit
// Enter the number of blocks:5
// Enter the number of processes:4
// Enter the size of the blocks:-
// Block no.1 : 100
// Block no.2 : 500
// Block no.3 : 200
// Block no.4 : 300
// Block no.5 : 600
// Enter the size of the processes :-
// Process no.1 : 212
// Process no.2 : 417
// Process no.3 : 112
// Process no.4 : 426
// Process_no      Process_size    Block_no        Block_size      Fragment
// 1               212             4               300             88
// 2               417             2               500             83
// 3               112             3               200             88
// 4               426             5               600             174

In [ ]:
// worst.c
#include <stdio.h>
int main() {
    int b, p, bsize[10], psize[10], all[10], flag[10];
    printf("Enter no of blocks : ");
    scanf("%d", &b);
    printf("Enter the size of each block : ");
    for (int i = 0; i < b; i++) scanf("%d", &bsize[i]);
    printf("Enter the no of processes : ");
    scanf("%d", &p);
    printf("Enter the size of each process : ");
    for (int i = 0; i < p; i++) scanf("%d", &psize[i]);
    for (int i = 0; i < p; i++) {
        flag[i] = 0;
        all[i] = -1;
    }
    for (int i = 0; i < p; i++) {
        int k = -1;
        for (int j = 0; j < b; j++) {
            if (bsize[j] >= psize[i] && !flag[j]) {
                if (k == -1 || bsize[k] < bsize[j]) k = j;
            }
        }
        if (k != -1) {
            all[i] = k;
            bsize[k] -= psize[i];
            flag[k] = 1;
        }
    }
    printf("\nProcess No.\tProcess Size\tBlock no.\n");
    for (int i = 0; i < p; i++) {
        printf("%d \t\t %d \t\t", i + 1, psize[i]);
        if (all[i] != -1) printf("%d\n", all[i] + 1);
        else printf("Not Allocated\n");
    }
}
// $ ./worst
// Enter no of blocks : 5
// Enter the size of each block : 100 500 200 300 600
// Enter the no of processes : 4
// Enter the size of each process : 212 417 112 426
// Process No      Process Size    Block no.
// 1                212            5
// 2                417            2
// 3                112            4
// 4                426           Not Allocated

PRODUCER CONSUMER


In [ ]:
// producer consumer problem using semaphores
#include <stdio.h>
#include <stdlib.h>
int mutex = 1, full = 0, empty = 3, x = 0;
int main() {
    int n;
    void producer();
    void consumer();
    int wait(int);
    int signal(int);
    printf("\n1.Producer\t2.Consumer\t3.Exit");
    while (1) {
        printf("\nEnter your choice:");
        scanf("%d", &n);
        switch (n) {
            case 1:
                if ((mutex == 1) && (empty != 0)) producer();
                else printf("Buffer is full");
                break;
            case 2:
                if ((mutex == 1) && (full != 0)) consumer();
                else printf("Buffer is empty");
                break;
            case 3:
                exit(0);
                break;
        }
    }
}
int wait(int s) { return (--s); }
int signal(int s) { return (++s); }
void producer() {
    mutex = wait(mutex);
    full = signal(full);
    empty = wait(empty);
    x++; printf("\nProducer produces the item %d", x);
    mutex = signal(mutex);
}
void consumer() {
    mutex = wait(mutex);
    full = wait(full);
    empty = signal(empty);
    printf("\nConsumer consumes item %d", x); x--;
    mutex = signal(mutex);
}
// $ ./pcs
// 1.Producer 2.Consumer 3.Exit
// Enter your choice:1
// Producer produces the item 1
// Enter your choice:1
// Producer produces the item 2
// Enter your choice:1
// Producer produces the item 3
// Enter your choice:2
// Consumer consumes item 3
// Enter your choice:1
// Producer produces the item 3
// Enter your choice:2
// Consumer consumes item 3
// Enter your choice:3

PAGING

In [ ]:
#include <stdio.h>
#define MAX 50
int main() {
    int page[MAX], i, n, f, ps, off, pno;
    int choice = 0;
    printf("Enter the no of pages in memory : "); scanf("%d", &n);
    printf("Enter page size : "); scanf("%d", &ps);
    printf("Enter no of frames : "); scanf("%d", &f);
    for (i = 0; i < n; i++) page[i] = -1;
    printf("Enter the page table\n(Enter frame no as -1 if that page is not present in any frame)");
    printf("\nPage no.\tFrame no.\n");
    for (i = 0; i < n; i++) {
        printf("%d\t\t", i);
        scanf("%d", &page[i]);
    }
    while(choice==1) {
        printf("Enter the logical address (i.e,page no & offset) : "); scanf("%d%d", &pno, &off);
        if (page[pno] == -1) printf("The required page is not available in any of frames\n");
        else printf("Physical address (frame no., offset) : %d, %d\n", page[pno], off);
        printf("Do you want to continue(1/0)? : "); scanf("%d", &choice);
    }
}
// Enter the no of pages in memory : 4
// Enter page size : 10
// Enter no of frames : 10
// Enter the page table
// (Enter frame no as -1 if that page is not present in any frame)
// Page no.        Frame no.
// 0               5
// 1               -1
// 2               8
// 3               2
// Enter the logical address (i.e,page no & offset) : 0 200
// Physical address (frame no., offset) : 5, 200
// Do you want to continue(1/0)? : 0